In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import datetime
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
training_data = pd.read_csv("organized_data_training.csv")

In [3]:
training_data_array = training_data.values

In [4]:
train_x = training_data_array[:, 4:7]
train_y = training_data_array[:,-1:]


In [5]:
# tt = train_x[:2]
# print(tt)

In [6]:
# print(tt[0][0])

In [7]:
# w_temp = np.array([[1,1,1]])
# ttt = w_temp @ tt.T
# print(np.tanh(ttt))

In [8]:
w_array = np.array([0,0,0])

In [9]:
# print(train_x[0])

In [10]:
# print(y_predict.shape)

In [11]:
# print(1 - y_predict[i]**2)

In [12]:
# print((1) * (1 - y_predict[i]**2) * train_x[i][0])

In [13]:
# w_array = np.array([1,1,1])

In [14]:
# print(w_array.shape)

In [15]:
# print(np.dot(w_array,train_x[0].T))

In [16]:
# print(train_x[:2].T)

In [17]:
# print(w_array@(train_x[:2].T))

In [18]:
def forward(w_array, x_array):
    temp = w_array @ x_array.T
    tanh_array = np.tanh(temp)
    return tanh_array

In [19]:
def calculate_gradient_descent(y_predict,y_true,w_array,x_array):
    gradients = []
    term1 = 0
    term2 = 0
    term3 = 0
    for i in range(len(y_predict)):
        temp1 = float((y_predict[i] - y_true[i]) * (1 - y_predict[i]**2) * x_array[i][0])
        temp2 = float((y_predict[i] - y_true[i]) * (1 - y_predict[i]**2) * x_array[i][1])
        temp3 = float((y_predict[i] - y_true[i]) * (1 - y_predict[i]**2) * x_array[i][2])
        term1 += temp1
        term2 += temp2
        term3 += temp3
    gradients = np.array([term1,term2,term3])
    return gradients

In [20]:
def update_gradient(gradients, w_array, lr):
    w_array = w_array - lr*gradients
    return w_array

In [21]:
# y_predict = forward(w_array, train_x)
# print(y_predict)

In [22]:
for i in range(3):
    y_predict = forward(w_array, train_x)
    gradients = calculate_gradient_descent(y_predict,train_y,w_array,train_x)
    w_array = update_gradient(gradients, w_array, 0.0001)
print(forward(w_array, train_x))

[ 0.08179755  0.0899231   0.09498796 ... -0.07222365 -0.07663649
 -0.08076924]


In [ ]:
y_predict = forward(w_array, train_x)
print(train_y - y_predict)